In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

Update with new data

# Week 13
- this is still astronomically smaller than the best point—over 70 orders of magnitude worse.
- This point lies inside the local search box around the best location. EI selected it because the GP thought there might be residual improvement in that direction.
- The outcome shows that the high-value region does not extend this way.
- Multiple nearby samples now fall off a cliff in value as in small spatial changes result in enormous output collapse
- This confirms the function has a needle-like maximum, not a ridge.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel as C, Matern

# -------------------------------------------------------
# 1. Existing samples (UPDATE THESE AS YOU GET NEW POINTS)
# -------------------------------------------------------
X_all = np.array([
    [0.31940389, 0.76295937],
    [0.57432921, 0.8798981 ],
    [0.73102363, 0.73299988],
    [0.84035342, 0.26473161],
    [0.65011406, 0.68152635],
    [0.41043714, 0.1475543 ],
    [0.31269116, 0.07872278],
    [0.68341817, 0.86105746],
    [0.08250725, 0.40348751],
    [0.88388983, 0.58225397],
    [0.080808,   0.404040],
    [0.393939,   0.070707],
    [0.020000,   0.020000],
    [0.349697, 0.136364], 
    [0.378788, 0.213939],
    [0.175152, 0.146061],
    [0.810127, 0.784810],
    [0.262017, 0.132941],
    [0.116807, 0.173277],
    [0.381963, 0.140775],
    [0.333126, 0.105201]
])

y_all = np.array([
    1.32267704e-079, 1.03307824e-046, 7.71087511e-016,
    3.34177101e-124, -3.60606264e-003, -2.15924904e-054,
    -2.08909327e-091, 2.53500115e-040, 3.60677119e-081,
    6.22985647e-048, 5.34214011784672e-82,
    4.676119097408122e-88, 9.127963232956071e-225, -3.3503867359112736e-61, 
    3.785787424178565e-33, -2.569999506751365e-97, 1.4616788805572485e-40,
    1.6009855182935657e-76, -4.135367082290168e-108, -3.0975365976197873e-59,
    -1.0997469316819375e-75
])

One last ultra-local probe
- Shrink radius to 0.03–0.04
- Reduce xi to 1e-4
- Take 1–2 more samples
- Purpose: estimate peak curvature and confirm exact location.

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import numpy as np
from scipy.stats import norm

# Signed log transform (unchanged)
def signed_log_transform(y):
    eps = 1e-30
    signs = np.sign(y)
    signs[signs == 0] = 1.0
    return signs * np.log10(np.abs(y) + eps)

y_trans = signed_log_transform(y_all)

kernel = C(1.0, (1e-3, 1e3)) * RBF(
    length_scale=0.07,
    length_scale_bounds=(0.02, 0.2)
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-10,
    normalize_y=False,
    n_restarts_optimizer=4
)

gp.fit(X_all, y_trans)


In [ ]:


import numpy as np
from scipy.stats import norm

# -------------------------------------------------------
# Assumptions:
# - gp is already fitted on (X_all, y_trans)
# - y_trans is the signed log10-transformed output
# - X_all contains all sampled points so far
# -------------------------------------------------------

# 1. Identify current best point (on transformed scale)
best_idx = np.argmax(y_trans)
x_best = X_all[best_idx]
y_best = y_trans[best_idx]

# -------------------------------------------------------
# 2. Generate LOCAL candidate region around best point (very tight)
# -------------------------------------------------------

radius = 0.035  # ultra-local refinement

x1 = np.linspace(
    max(0.0, x_best[0] - radius),
    min(1.0, x_best[0] + radius),
    150
)

x2 = np.linspace(
    max(0.0, x_best[1] - radius),
    min(1.0, x_best[1] + radius),
    150
)

X_candidates = np.array([[i, j] for i in x1 for j in x2])

# -------------------------------------------------------
# 3. Expected Improvement acquisition (very low ξ)
# -------------------------------------------------------
def acquisition_ei(X, gp, y_best, xi=1e-4): # down from xi=0.01
    mu, sigma = gp.predict(X, return_std=True)
    sigma = np.maximum(sigma, 1e-9)

    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-8] = 0.0
    return ei

# -------------------------------------------------------
# 4. Select next point
# -------------------------------------------------------
ei_values = acquisition_ei(X_candidates, gp, y_best, xi=1e-4)

best_idx = np.argmax(ei_values)
next_point = X_candidates[best_idx]
best_ei = ei_values[best_idx]

print(f"Next point to query: [{next_point[0]:.6f}, {next_point[1]:.6f}]")
print(f"EI value: {best_ei:.6e}")


# ei_values = acquisition_ei(X_candidates, gp, y_best, xi=0.001)

# best_idx = np.argmax(ei_values)
# next_point = X_candidates[best_idx]
# best_ei = ei_values[best_idx]

# print(f"Next point to query: {next_point}")
# print(f"EI value: {best_ei:.6f}")

# print("Next EI-refinement point(rounded):", np.round(next_point, 6))


In [ ]:
print(x_best)

# Final Result
- Best observed value: ≈ −3.6 × 10⁻³
- Typical values during global exploration: 10⁻³⁰ → 10⁻¹²⁴
- Recent local EI probes: 10⁻⁵⁰ → 10⁻¹⁰⁸
- This point: 10⁻⁵⁵
- So this point is ~52 orders of magnitude worse than the best.
- This location is still near the best region spatially, but clearly outside the narrow basin of attraction.